In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')


In [25]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from llama_index.core import SimpleDirectoryReader
doc = PyPDFDirectoryLoader(r"C:\Users\Win10\Desktop\data").load()
doc

[Document(metadata={'source': 'C:\\Users\\Win10\\Desktop\\data\\9637_chap01.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/301253073\nIntroduction to Solar Energy\nChapter · June 2016\nDOI: 10.1142/9789814689502_0001\nCITATIONS\n18\nREADS\n35,662\n3 authors, including:\nRichard Corkish\nUNSW Sydney\n99 PUBLICATIONS\xa0\xa0\xa03,289 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Richard Corkish on 20 January 2021.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'C:\\Users\\Win10\\Desktop\\data\\9637_chap01.pdf', 'page': 1}, page_content='Chapter 1\nIntroduction to Solar Energy\nR. Corkish,∗,† W. Lipi´nski‡ and R. J. Patterson∗\n∗University of New South Wales, Sydney 2052, Australia\n†Australian Centre for Advanced Photovoltaics, Sydney 2052, Australia\n‡The Australian National University, Canberra 2601, Australia

In [ ]:

doc[0].page_content
# doc

'See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/301253073\nIntroduction to Solar Energy\nChapter · June 2016\nDOI: 10.1142/9789814689502_0001\nCITATIONS\n18\nREADS\n35,662\n3 authors, including:\nRichard Corkish\nUNSW Sydney\n99 PUBLICATIONS\xa0\xa0\xa03,289 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Richard Corkish on 20 January 2021.\nThe user has requested enhancement of the downloaded file.'

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators=["\n","\n\n"],chunk_size=500, chunk_overlap=50)

In [42]:
from langchain.docstore.document import Document
chunking_docs = []
for i in doc:
    chunks = text_splitter.split_text(i.page_content)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk,metadata=i.metadata)
        chunking_docs.append(chunk_doc)

In [44]:
chunking_docs

[Document(metadata={'source': 'C:\\Users\\Win10\\Desktop\\data\\9637_chap01.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/301253073\nIntroduction to Solar Energy\nChapter · June 2016\nDOI: 10.1142/9789814689502_0001\nCITATIONS\n18\nREADS\n35,662\n3 authors, including:\nRichard Corkish\nUNSW Sydney\n99 PUBLICATIONS\xa0\xa0\xa03,289 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Richard Corkish on 20 January 2021.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'C:\\Users\\Win10\\Desktop\\data\\9637_chap01.pdf', 'page': 1}, page_content='Chapter 1\nIntroduction to Solar Energy\nR. Corkish,∗,† W. Lipi´nski‡ and R. J. Patterson∗\n∗University of New South Wales, Sydney 2052, Australia\n†Australian Centre for Advanced Photovoltaics, Sydney 2052, Australia\n‡The Australian National University, Canberra 2601, Australia

In [12]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
llm = OpenAI()


In [13]:
llm.invoke("what is machine learning")  

'\nMachine learning is a subset of artificial intelligence that involves building algorithms and models that enable computers to learn from data without being explicitly programmed. These algorithms and models use statistical techniques to identify patterns and make predictions or decisions based on the data. Machine learning is used in a wide range of applications, such as image and speech recognition, natural language processing, and predictive analytics. It is also a key component in many emerging technologies, such as self-driving cars and virtual assistants.'

In [77]:
import chromadb
embede_model = OpenAIEmbeddings(model="text-embedding-ada-002")
db_client = chromadb.PersistentClient(path="pdf_vector")
collection_name = "my_pdf_data"
# collection = db_client.create_collection(name=collection_name, dimension=1536)
if collection_name not in [col.name for col in db_client.list_collections()]:
    collection = db_client.create_collection(name=collection_name)
else:
    collection = db_client.get_collection(collection_name)




In [78]:
# db_client.delete_collection(name="my_pdf_data")
# # Check if the collection already exists with the correct dimensions
# if not db_client.get_collection(collection_name):
#     collection = db_client.create_collection(name=collection_name, dimension=1536)
#     print("created 1536 dimenstion")
# else:
#     collection = db_client.get_collection(collection_name)
collection.count()

0

In [79]:
import uuid
for chunking_doc in chunking_docs:
    # embedding = embede_model.embed_query(chunking_doc.page_content)
    # print(embedding)
    collection.add(
        documents=[chunking_doc.page_content],
        # embeddings=[embedding],
        ids=[str(uuid.uuid4())],
        metadatas=[chunking_doc.metadata]
    )

In [83]:
embedding

[-0.014664667213034997,
 0.010964554166468141,
 0.02411522982443603,
 -0.023069693539920725,
 -0.043858216665872564,
 0.026790174611869225,
 -0.03323991334295678,
 -0.01234955102141931,
 -0.03943165894852201,
 -0.024576895753193953,
 -0.007597112348543266,
 0.035466769022574604,
 -0.021562491326647493,
 0.0020520350178690433,
 -0.009498088003958204,
 -0.008255664532710743,
 -0.0033742313925494084,
 0.015126332210470318,
 0.024753414914544462,
 0.007176182004887331,
 -0.024536161565075872,
 0.023083272223508488,
 -0.01917269516926692,
 -0.029736685620476863,
 0.011005289285908828,
 0.0013349260412979166,
 0.007861891090569033,
 -0.01618544810989599,
 -0.010068380607450425,
 0.003004219948194332,
 0.028677570652373797,
 -0.01116823069499418,
 -0.01331361706705324,
 -0.0019434078774734442,
 -0.010591148749708077,
 0.00948450932037044,
 -0.015506527900346867,
 -0.019593625047261552,
 0.009518455098017246,
 0.003961497117695683,
 0.012879108505470842,
 6.778589383363574e-05,
 0.002191213497

In [98]:
query_result = collection.query(query_texts="who is author of solar energy publication",n_results=2)


In [99]:
query_result

{'ids': [['07572caf-7c04-4bf5-87db-95b07c25cb06',
   '0b65ea4a-49bd-4e99-83af-3ee825147010']],
 'embeddings': None,
 'documents': [['See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/301253073\nIntroduction to Solar Energy\nChapter · June 2016\nDOI: 10.1142/9789814689502_0001\nCITATIONS\n18\nREADS\n35,662\n3 authors, including:\nRichard Corkish\nUNSW Sydney\n99 PUBLICATIONS\xa0\xa0\xa03,289 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Richard Corkish on 20 January 2021.\nThe user has requested enhancement of the downloaded file.',
   '(Oxford University Press, Oxford, 1992).\n30. B. Sorensen, Energy Intermittency (CRC Press, Boca Raton, 2014).\n31. J. C. Mankins, in The First International Assessment of Space Solar Power:\nOpportunities, Issues and Potential Ways Forward (International Academy\nof Astronautics, Stockholm, 2011).\n32. Lazard, Lazard’s Levelized Cost of Energy Analysis — Ve

In [ ]:

result = llm.invoke(input=query_result.get('documents'))
# print(result)

ValueError: Unexpected message type: 'See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/301253073
Introduction to Solar Energy
Chapter · June 2016
DOI: 10.1142/9789814689502_0001
CITATIONS
18
READS
35,662
3 authors, including:
Richard Corkish
UNSW Sydney
99 PUBLICATIONS   3,289 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Richard Corkish on 20 January 2021.
The user has requested enhancement of the downloaded file.'. Use one of 'human', 'user', 'ai', 'assistant', 'function', 'tool', or 'system'.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE